In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [ ]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

In [ ]:
from src.et_green.compute_et_green import compute_et_green, calculate_band_std_dev
from src.et_green.filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)
from src.et_green.exporting_utils import process_et_green, prepare_rainfed_fields

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
    is_image_empty,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [ ]:
YEAR = 2023

PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/ET_Landsat_decadal_canton_v2"
PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{YEAR}"
PATH_TO_JURISDICTION = (
    f"projects/thurgau-irrigation/assets/Thurgau/thurgau_jurisdictions_{YEAR}_wgs84"
)
PATH_TO_LANDUSE = (
    f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{YEAR}_area"
)
LANDUSE_PROPERTY_NAME = "nutzung"

ET_BAND_NAME = "ET"
SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
ET_BAND_RESOLUTION = 30  # in meters
NUMBER_OF_IMAGES = 36
TEMPORAL_RESOLUTION = "dekadal"


ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_{YEAR}_{TEMPORAL_RESOLUTION}_from_Landsat_30m"

## 1. Load Assets

In [ ]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [ ]:
double_cropping_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)

In [ ]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    .filter(ee.Filter.eq("Region", "Thurgau"))
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())

# Sanity checks:
print_collection_dates(et_collection)
print(f"Sizing of the ET collection: {et_collection.size().getInfo()}")

In [ ]:
jurisdictions = ee.FeatureCollection(PATH_TO_JURISDICTION)

In [ ]:
landuse_collection = ee.FeatureCollection(PATH_TO_LANDUSE)

# Check if the collection has "nutzung" property, otherwise rename it
properties = landuse_collection.first().propertyNames().getInfo()

landuse_collection = (
    landuse_collection.map(lambda f: f.set("nutzung", f.get(LANDUSE_PROPERTY_NAME)))
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else landuse_collection
)

print(
    f"Renamed {LANDUSE_PROPERTY_NAME} to 'nutzung'"
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else "Collection has 'nutzung' property"
)

## 2. Compute ET_green

In [ ]:
# process_et_green(
#     et_collection_list=et_collection_list,
#     landuse_collection=landuse_collection,
#     jurisdictions=jurisdictions,
#     double_cropping_image=double_cropping_image,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_GREEN_ASSET_PATH,
#     et_band_name=ET_BAND_NAME,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=ET_BAND_RESOLUTION,
# )

In [ ]:
et_green_list = []


not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

# Prepare rainfed fields
rainfed_fields = prepare_rainfed_fields(
    landuse_collection, double_cropping_image, not_irrigated_crops, rainfed_crops
)

for img in range(NUMBER_OF_IMAGES):
    et_img = ee.Image(et_collection_list.get(img))

    et_green_img = compute_et_green(
        et_img, rainfed_fields, jurisdictions, et_band_name=ET_BAND_NAME
    )
    et_green_list.append(et_green_img)

In [ ]:
# for image in et_green_list:
#     print(f"Date: {ee.Image(image).date().format('YYYY-MM-dd').getInfo()}")

# # print(f"Number of images: {len(et_green_list)}")

In [ ]:
Map = geemap.Map()

image = ee.Image(et_green_list[15])
image_ET = ee.Image(et_collection_list.get(15))

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 5,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

vis_params_ET = {
    "bands": [ET_BAND_NAME],
    "min": 0,
    "max": 5,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}



Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET green 10m")
Map.add_colorbar(vis_params, label="ET green [mm/month]")
Map.addLayer(image_ET, vis_params_ET, "ET 10m")


Map

### Comparing WaPOR to Landsat

In [ ]:
# landsat_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_2018_dekadal_from_Landsat_30m").map(lambda img: back_to_float(img, 100))

# wapor_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_2018_dekadal_from_WaPOR_10m").map(lambda img: back_to_float(img, 100))

In [ ]:
# print_collection_dates(landsat_collection)
# print_collection_dates(wapor_collection)

In [ ]:
# landsat_list = landsat_collection.toList(landsat_collection.size())
# wapor_list = wapor_collection.toList(wapor_collection.size())

In [ ]:
# landsat_august = ee.Image(landsat_list.get(12))
# wapor_august = ee.Image(wapor_list.get(21))

# Map = geemap.Map()

# vis_params = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 5,
#     "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
# }

# Map.center_object(aoi, 12)
# Map.addLayer(landsat_august, vis_params, "ET Landsat")
# Map.addLayer(wapor_august, vis_params, "ET WaPOR")

# # Print the date of the images
# print(f"Landsat image date: {ee.Image(landsat_august).date().format('YYYY-MM-dd').getInfo()}")
# print(f"WaPOR image date: {ee.Image(wapor_august).date().format('YYYY-MM-dd').getInfo()}")

# Map

NameError: name 'ee' is not defined